## 循环神经网络（RNN）

#### Sidenote
- Hidden是现实中存在，但是我们没观察到
- Latent是现实中不存在的，可和hidden互换

### 潜变量自回归模型
![](./imgs/54-1.png)

### 循环神经网络
- RNN与MLP的区别是: 用额外的$W_{hh}$来存时序信息
![](./imgs/54-2.png)
- 举例说明
![](./imgs/54-3.png)

### 困惑度(perplexity)
- 衡量一个语言模型的好坏可以用平均交叉熵
![](./imgs/54-4.png)
- 历史原因NLP使用困惑度$exp(\pi)$来衡量(平均交叉熵取指数),表示每一次的可能选项
- 1表示完美，无穷是最大的情况

### 梯度剪裁（Gradient Clip）
- 在迭代中，计算这T个时间步上的梯度，在反向传播过程中，会产生长度为O(T)的矩阵乘法链，会导致数值不稳定
- 梯度剪裁能有效预防梯度爆炸
- 下式中的g表示所有层的梯度，放在一个向量里
- 这个式子表达的意思是，假设g的长度大于theta，则投影回theta
![](./imgs/54-5.png)

### 更多RNN的应用
![](./imgs/54-6.png)

### 总结
- RNN的输出取决于当前输入和前一时间的隐变量
- 使用perplexity来衡量语言模型的好坏
- 应用到语言模型中，RNN根据当前时刻词预测下一时刻词

## RNN从零开始实现

In [2]:
%matplotlib inline
import math
import torch
from torch import nn
from torch.nn import functional as F
import d2l

In [3]:
batch_size, num_steps = 32, 35
train_iter, vocab = d2l.load_data_time_machine(batch_size, num_steps)
# vocal的用处是整数和词的转化

In [6]:
# index -> one-hot
F.one_hot(torch.tensor([0, 2]), len(vocab)) # [0, 2]

tensor([[1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0],
        [0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
         0, 0, 0, 0]])

In [7]:
X = torch.arange(10).reshape((2, 5))  # 最后一个维度是时间
F.one_hot(X.T, 28).shape # 这里需要做转置，X.T: [T,B,L]

torch.Size([5, 2, 28])

In [8]:
def get_params(vocab_size, num_hiddens, device):
    num_inputs = num_outputs = vocab_size # 因为经过了onehot，所以输入长度等于vocab_size
    
    # 一个辅助函数: 初始化函数
    def normal(shape):
        return torch.randn(size=shape, device=device) * 0.01

    # 隐藏层参数
    W_xh = normal((num_inputs, num_hiddens))
    W_hh = normal((num_hiddens, num_hiddens)) # 挪掉这一行就是MLP
    b_h = torch.zeros(num_hiddens, device=device)
    # 输出层参数
    W_hq = normal((num_hiddens, num_outputs))
    b_q = torch.zeros(num_outputs, device=device)
    
    # 附加梯度
    params = [W_xh, W_hh, b_h, W_hq, b_q]
    for param in params:
        param.requires_grad_(True)
    return params

In [9]:
# 初始化隐藏变量: 因为0时刻没有隐藏变量，需要自己初始化
def init_rnn_state(batch_size, num_hiddens, device):
    # 隐藏状态放在一个tuple里(之后会有多个隐藏状态)
    return (torch.zeros((batch_size, num_hiddens), device=device), ) # 也可以换成随机

In [10]:
def rnn(inputs, state, params):
    # `inputs`的形状：(`time_step数量`，`批量大小`，`词表大小`)
    # input里其实有time_step个样本
    W_xh, W_hh, b_h, W_hq, b_q = params
    H, = state
    outputs = []
    # `X`的形状：(`批量大小`，`词表大小`)
    for X in inputs:
        H = torch.tanh(torch.mm(X, W_xh) 
                       + torch.mm(H, W_hh) 
                       + b_h)
        Y = torch.mm(H, W_hq) + b_q
        outputs.append(Y) 
    return torch.cat(outputs, dim=0), (H,) # 

In [11]:
class RNNModelScratch: #
    """从零开始实现的循环神经网络模型"""
    def __init__(self, vocab_size, num_hiddens, device,
                 get_params, init_state, forward_fn):
        self.vocab_size, self.num_hiddens = vocab_size, num_hiddens
        self.params = get_params(vocab_size, num_hiddens, device)
        self.init_state, self.forward_fn = init_state, forward_fn

    def __call__(self, X, state):
        X = F.one_hot(X.T, self.vocab_size).type(torch.float32)
        return self.forward_fn(X, state, self.params)
    
    # 初始状态
    def begin_state(self, batch_size, device):
        return self.init_state(batch_size, self.num_hiddens, device)

In [ ]:
# 检查输出是否正确
num_hiddens = 512
net = RNNModelScratch(len(vocab), num_hiddens, d2l.try_gpu(), get_params,
                      init_rnn_state, rnn)
state = net.begin_state(X.shape[0], d2l.try_gpu())
Y, new_state = net(X.to(d2l.try_gpu()), state)
Y.shape, len(new_state), new_state[0].shape

In [ ]:
def predict_ch8(prefix, num_preds, net, vocab, device):  #@save
    """在`prefix`后面生成新字符。"""
    state = net.begin_state(batch_size=1, device=device)
    outputs = [vocab[prefix[0]]]
    get_input = lambda: torch.tensor([outputs[-1]], device=device).reshape((1, 1))
    for y in prefix[1:]:  # 预热期
        _, state = net(get_input(), state)
        outputs.append(vocab[y])
    for _ in range(num_preds):  # 预测`num_preds`步
        y, state = net(get_input(), state)
        outputs.append(int(y.argmax(dim=1).reshape(1)))
    return ''.join([vocab.idx_to_token[i] for i in outputs])

In [ ]:
predict_ch8('time traveller ', 10, net, vocab, d2l.try_gpu())

In [ ]:
def grad_clipping(net, theta):  #@save
    """裁剪梯度。"""
    if isinstance(net, nn.Module):
        params = [p for p in net.parameters() if p.requires_grad]
    else:
        params = net.params
    norm = torch.sqrt(sum(torch.sum((p.grad ** 2)) for p in params))
    if norm > theta:
        for param in params:
            param.grad[:] *= theta / norm

In [ ]:
#@save
def train_epoch_ch8(net, train_iter, loss, updater, device, use_random_iter):
    """训练模型一个迭代周期（定义见第8章）。"""
    state, timer = None, d2l.Timer()
    metric = d2l.Accumulator(2)  # 训练损失之和, 词元数量
    for X, Y in train_iter:
        if state is None or use_random_iter:
            # 在第一次迭代或使用随机抽样时初始化`state`
            state = net.begin_state(batch_size=X.shape[0], device=device)
        else:
            if isinstance(net, nn.Module) and not isinstance(state, tuple):
                # `state`对于`nn.GRU`是个张量
                state.detach_()
            else:
                # `state`对于`nn.LSTM`或对于我们从零开始实现的模型是个张量
                for s in state:
                    s.detach_()
        y = Y.T.reshape(-1)
        X, y = X.to(device), y.to(device)
        y_hat, state = net(X, state)
        l = loss(y_hat, y.long()).mean()
        if isinstance(updater, torch.optim.Optimizer):
            updater.zero_grad()
            l.backward()
            grad_clipping(net, 1)
            updater.step()
        else:
            l.backward()
            grad_clipping(net, 1)
            # 因为已经调用了`mean`函数
            updater(batch_size=1)
        metric.add(l * y.numel(), y.numel())
    return math.exp(metric[0] / metric[1]), metric[1] / timer.stop()

In [ ]:
#@save
def train_ch8(net, train_iter, vocab, lr, num_epochs, device,
              use_random_iter=False):
    """训练模型（定义见第8章）。"""
    loss = nn.CrossEntropyLoss()
    animator = d2l.Animator(xlabel='epoch', ylabel='perplexity',
                            legend=['train'], xlim=[10, num_epochs])
    # 初始化
    if isinstance(net, nn.Module):
        updater = torch.optim.SGD(net.parameters(), lr)
    else:
        updater = lambda batch_size: d2l.sgd(net.params, lr, batch_size)
    predict = lambda prefix: predict_ch8(prefix, 50, net, vocab, device)
    # 训练和预测
    for epoch in range(num_epochs):
        ppl, speed = train_epoch_ch8(
            net, train_iter, loss, updater, device, use_random_iter)
        if (epoch + 1) % 10 == 0:
            print(predict('time traveller'))
            animator.add(epoch + 1, [ppl])
    print(f'困惑度 {ppl:.1f}, {speed:.1f} 词元/秒 {str(device)}')
    print(predict('time traveller'))
    print(predict('traveller'))

In [ ]:
num_epochs, lr = 500, 1
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu())

In [1]:
train_ch8(net, train_iter, vocab, lr, num_epochs, d2l.try_gpu(),
          use_random_iter=True)

NameError: name 'train_ch8' is not defined